In [3]:
import yfinance as yf 
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

In [4]:
df = web.DataReader("AAPL", data_source="yahoo", start="2012-01-01", end="2019-12-17")

In [5]:
data = df.filter(["Close"])

In [6]:
dataset = data.values

In [7]:
training_data_len = math.ceil(len(dataset)*.8)

In [8]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.01316509],
       [0.01457064],
       [0.01748985],
       ...,
       [0.97658263],
       [0.99755134],
       [1.        ]])

In [9]:
train_data = scaled_data[0:training_data_len, :]

In [55]:
x_train = []
y_train = []

for i in range(10, len(train_data)): 
    x_train.append(train_data[i-10:i, 0]) #Los valores de 1 a 60
    y_train.append(train_data[i, 0]) #Contiene el valor 61 -> valor a predecir
    if i <= 11:
        print(x_train)
        print(y_train)

[array([0.01316509, 0.01457064, 0.01748985, 0.02026915, 0.01984303,
       0.02080338, 0.02036454, 0.01962679, 0.01862193, 0.02173194])]
[0.02453667868228926]
[array([0.01316509, 0.01457064, 0.01748985, 0.02026915, 0.01984303,
       0.02080338, 0.02036454, 0.01962679, 0.01862193, 0.02173194]), array([0.01457064, 0.01748985, 0.02026915, 0.01984303, 0.02080338,
       0.02036454, 0.01962679, 0.01862193, 0.02173194, 0.02453668])]
[0.02453667868228926, 0.023671723005769474]


In [56]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [57]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [58]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [59]:
model.compile(optimizer = "adam", loss="mean_squared_error")

In [60]:
model.fit(x_train, y_train, batch_size = 1, epochs = 1)

1593/1593 [==============================] - 13s 8ms/step - loss: 7.0233e-04


TEST

In [61]:
test_data = scaled_data[training_data_len - 200: , :] 
#Cojo el trozo que no he cogido en el training
x_test = []
y_test = dataset[training_data_len:, :] #Valores que queremos que prediga nuestro modelo
for i in range(200, len(test_data)):
    x_test.append(test_data[i-200:i, 0])

In [102]:
last_10_days = data[-10:].values 
#Creo una array de los últimos 200 días 
last_10_days_scaled = scaler.transform(last_10_days)
#Los escalo
X_test = []
#Creo una lista
X_test.append(last_10_days_scaled)
print(X_test)
#Le meto los valores de los 200 dias escalados
X_test = np.array(X_test)
print(X_test)
#Lo convierto en array
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Sacamos la predicción 
pred_price = model.predict(X_test)
#Esta línea lo que hace es aplicar el modelo que acabamos de hacer a los 200 días que le damos, para sacar el valor del día 201
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

[array([[0.91688179],
       [0.93397731],
       [0.9568159 ],
       [0.93994306],
       [0.94688811],
       [0.95708301],
       [0.96015487],
       [0.97658263],
       [0.99755134],
       [1.        ]])]
[[[0.91688179]
  [0.93397731]
  [0.9568159 ]
  [0.93994306]
  [0.94688811]
  [0.95708301]
  [0.96015487]
  [0.97658263]
  [0.99755134]
  [1.        ]]]
[[70.958176]]


In [104]:
last_10_days = data[-10:].values 
last_10_days_scaled = scaler.transform(last_10_days)
X_test = []
X_test.append(last_10_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
for i in range(0, 10):
    counter = 0
    pred_price = model.predict(X_test)
    X_test = np.append(X_test, pred_price)
    X_test = np.array(X_test)
    print(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    counter =+ 1
    print("o")
    print(X_test)
    if counter == 10:
        break
   

[0.91688179 0.93397731 0.9568159  0.93994306 0.94688811 0.95708301
 0.96015487 0.97658263 0.99755134 1.         1.01523769]


IndexError: tuple index out of range

In [106]:
last_10_days = data[-10:].values 
last_10_days_scaled = scaler.transform(last_10_days)
X_test = []


array([0.91688179, 0.93397731, 0.9568159 , 0.93994306, 0.94688811,
       0.95708301, 0.96015487, 0.97658263, 0.99755134, 1.        ,
       1.01523769])

In [85]:
X_test

array([ 0.91688179,  0.93397731,  0.9568159 ,  0.93994306,  0.94688811,
        0.95708301,  0.96015487,  0.97658263,  0.99755134,  1.        ,
       70.95817566])

In [65]:
X_test2 = np.append(X_test, pred_price)

In [49]:
X_test2

array([ 0.52858161,  0.51958863,  0.52141395,  0.54808117,  0.55702966,
        0.56059124,  0.56958416,  0.58022435,  0.58868312,  0.58204967,
        0.58930639,  0.62015846,  0.60217256,  0.59188854,  0.58320715,
        0.59068649,  0.59179948,  0.59727537,  0.60301844,  0.6153949 ,
        0.62131602,  0.62282967,  0.62866173,  0.64246284,  0.63979164,
        0.64477781,  0.63734304,  0.63698688,  0.63858959,  0.63867865,
        0.65595228,  0.6592022 ,  0.66218501,  0.67531828,  0.67389369,
        0.66552398,  0.66116107,  0.66254117,  0.64500043,  0.68885229,
        0.68275305,  0.69432819,  0.67977025,  0.65475024,  0.65492828,
        0.64522304,  0.62946305,  0.57844359,  0.59153237,  0.60159378,
        0.59785415,  0.59304602,  0.5667349 ,  0.58236134,  0.56535481,
        0.5514647 ,  0.54839284,  0.54509836,  0.54131423,  0.54541003,
        0.53103021,  0.52315021,  0.55137564,  0.56428631,  0.57621761,
        0.59816575,  0.60898406,  0.61891192,  0.61615172,  0.61